# 导入库

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from textblob import TextBlob
import tensorflow.python.keras.backend as K
from wordcloud import WordCloud
import seaborn as sns
import numpy as np
import os, re
from tensorflow.python.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.python.keras.models import Model
# from transformers import *
from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold, KFold
import tensorflow_hub as hub
# import sentencepiece
# import tokenization
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text
from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'tensorflow.python.keras.backend'

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()


# chinese model

In [ ]:
from keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Activation, add,MultiHeadAttention
from keras_self_attention import SeqSelfAttention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
def create_custom_objects():
    instance_holder = {"instance": None}

    class ClassWrapper(AttentionWithContext):
        def __init__(self, *args, **kwargs):
            instance_holder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)

    def loss(*args):
        method = getattr(instance_holder["instance"], "loss_function")
        return method(*args)

    def accuracy(*args):
        method = getattr(instance_holder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"AttentionWithContext": ClassWrapper, "loss": loss,
            "accuracy":accuracy}
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
        
#     def get_config(self):
#         config = super().get_config()
#         config.update({
#             "arg1": self.arg1,
#             "arg2": self.arg2,
#         })
#         return config     
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
                'W_regularizer': self.W_regularizer,
                'u_regularizer': self.u_regularizer,
                'b_regularizer': self.b_regularizer,
                'W_constraint': self.W_constraint,
                'u_constraint': self.u_constraint,
                'b_constraint': self.b_constraint,
                'bias': self.bias,
        })
        return config

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]
class ParallelCoAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(ParallelCoAttention, self).__init__()
        self.W_text = tf.keras.layers.Dense(units)
        self.W_conv = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, sequence_output, conv_output):
        # 计算文本序列和卷积输出之间的相似性
        score_text = self.V(tf.nn.tanh(self.W_text(sequence_output)))
        score_conv = self.V(tf.nn.tanh(self.W_conv(conv_output)))
        
        # 计算注意力权重
        attention_weights_text = tf.nn.softmax(score_text, axis=1)
        attention_weights_conv = tf.nn.softmax(score_conv, axis=1)

        # 对文本序列和卷积输出进行注意力加权
        context_text = tf.reduce_sum(attention_weights_text * sequence_output, axis=1)
        context_conv = tf.reduce_sum(attention_weights_conv * conv_output, axis=1)

        return context_text, context_conv

In [ ]:
#hugging face 代理使用
from transformers import BertTokenizer, BertModel
import os
os.environ['CURL_CA_BUNDLE'] = ''
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = TFBertModel.from_pretrained("bert-base-chinese")

In [ ]:
df = pd.read_csv('./textrankfinal.csv',index_col=0)  #Zhccvi

In [ ]:
data = np.load('zhoutput_tensors.npy')
labels = np.load('zhlabels.npy')

In [ ]:
test_size = 0.2
# 使用train_test_split进行划分
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=test_size, random_state=42)

In [ ]:
from tensorflow.keras.layers import Concatenate, Dense, Dropout, Conv1D, MaxPooling1D, Reshape
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
f1_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []

In [ ]:
def build_bertCA():
    text_input = tf.keras.layers.Input(shape=(256, 768), dtype=tf.float32) 
    # 扩展维度以适应 AttentionWithContext 的要求
    attention_output = AttentionWithContext()(text_input)
    # 假设conv_output是卷积层的输出
    conv_output = Conv1D(filters=32, kernel_size=3, padding="valid", activation='relu')(text_input)
    conv_output = MaxPooling1D(pool_size=2)(conv_output)
    conv_output = GlobalAveragePooling1D()(conv_output)
    # 定义并行注意力机制层
    co_attention = ParallelCoAttention(units=64)
    context_text, context_conv = co_attention(conv_output, attention_output)
    context_text = tf.expand_dims(context_text, axis=1)  # 扩展维度
    context_conv = tf.expand_dims(context_conv, axis=1)  # 扩展维度
    pooler_output = tf.reduce_mean(text_input, axis=1)
    merged = tf.keras.layers.concatenate([context_text, context_conv,pooler_output], axis=1)
    fc1 = Dense(64, activation='relu', )(merged)
    dropout1 = Dropout(0.3)(fc1)
    fc2 = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))(dropout1)
    dropout2 = Dropout(0.3)(fc2)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(dropout2)
    return tf.keras.Model(text_input, net)

In [ ]:
num_folds = 5
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
learning_rate = 2e-5
learning_decay = 1e-4 # 1e-4
for train_index, test_index in kf.split(data, labels):
    train_data, test_data = data[train_index], data[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]
    callbacks = [EarlyStopping(monitor='val_loss', patience=2,mode="min",)]#当验证集的连续两个epoch指标不再改善时停止训练
    # Build a new BERT model for each fold
    bertCA = build_bertCAko()
    bertCA.compile(tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=learning_decay), loss='binary_crossentropy', metrics=['accuracy'])
    bertCA.fit(train_data, train_labels, epochs=10, batch_size=16,validation_split=0.1,callbacks=callbacks, verbose=1)
    predictions = bertCA.predict(test_data)
    binary_predictions = (predictions > 0.5).astype(int)
    f1 = f1_score(test_labels, binary_predictions)
    precision = precision_score(test_labels, binary_predictions)
    recall = recall_score(test_labels, binary_predictions)
    accuracy = accuracy_score(test_labels, binary_predictions)
    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)
    accuracy_scores.append(accuracy)
# Calculate the average accuracy across all folds

In [ ]:
average_accuracy = sum(accuracy_scores) / num_folds
average_f1_scores = sum(f1_scores) / num_folds
average_precision_scores = sum(precision_scores) / num_folds
average_recall_scores = sum(recall_scores) / num_folds

print(f'Average Accuracy: {average_accuracy}')
print(f'Average f1_scores: {average_f1_scores}')
print(f'Average precision_scores: {average_precision_scores}')
print(f'Average recall_scores: {average_recall_scores}')